This notebook will 
- dedup cs_faculty_top6-2023-09-04.xlsx
- update it with data/cs-faculty-gscholar-extra2.xlsx
- add extra column="not_found_date" to indicate no google scholar profile found on that date of search

In [1]:
from scrap_cs_faculty import *

## read dataset to enrich

In [2]:
file_xlsx = "cs_faculty_top6-2023-09-04.xlsx"
file_xlsx_extra = "data/cs-faculty-gscholar-extra2.xlsx"

In [3]:
df = pd.read_excel(Path(file_xlsx))

df.fillna("", inplace=True)

In [4]:
df.columns 

Index(['name', 'affiliation', 'interests', 'num_papers', 'num_coauthors',
       'citedby', 'hindex', 'i10index', 'citedby5y', 'hindex5y', 'i10index5y',
       'phd_univ', 'phd_year', 'url_homepage', 'url', 'url_author',
       'research_area', 'research_concentration', 'research_focus',
       'job_title', 'url_picture', 'img_url', 'phone', 'email', 'cell_phone',
       'office_address', 'department', 'school', 'scholar_id', 'file_author'],
      dtype='object')

In [5]:
sql_stmt = f"""
select distinct * from df 
--where scholar_id is null or scholar_id = ''
"""
df_u = duckdb.sql(sql_stmt).df()
df_u 

,name,affiliation,interests,num_papers,num_coauthors,citedby,hindex,i10index,citedby5y,hindex5y,...,url_picture,img_url,phone,email,cell_phone,office_address,department,school,scholar_id,file_author
0,Jiawei Han,"Abel Bliss Professor of Computer Science, Univ...",data mining; database systems; data warehousin...,1653.0,92.0,244378.0,200.0,1022.0,82746.0,117.0,...,https://scholar.google.com/citations?view_op=m...,https://ws.engr.illinois.edu/directory/viewpho...,(217) 333-6903,hanj@illinois.edu,,,Computer Science,Univ Illinois Urbana-Champaign,Kv9AbjMAAAAJ,data/GScholar_jiawei_han_uiuc.json
1,Robert Miller,"Professor, Computer Science, MIT",HCI; software engineering; crowdsourcing,1491.0,0.0,35488.0,78.0,324.0,13243.0,49.0,...,https://scholar.google.com/citations?view_op=m...,https://www.eecs.mit.edu/people/rob-miller/,(617) 324-6028,rcm@mit.edu,,Office: 32-G718,Computer Science,Massachusetts Institute Technology,e-c3R8QAAAAJ,data/GScholar_robert_miller_mit.json
2,"Kanade, Takeo",Carnegie Mellon University,Computer Vision,1302.0,0.0,147292.0,169.0,636.0,33531.0,74.0,...,https://scholar.google.com/citations?view_op=m...,,(412) 268-3016,kanade@andrew.cmu.edu,,,Computer Science,Carnegie Mellon Univ,LQ87h3sAAAAJ,data/GScholar_kanade_takeo_cmu.json
3,"Faloutsos, Christos",CMU,Data Mining; Graph Mining; Databases,1126.0,51.0,112459.0,148.0,589.0,34139.0,87.0,...,https://scholar.google.com/citations?view_op=m...,,(412) 268-1457,christos@andrew.cmu.edu,,7003 Gates and Hillman Centers,Computer Science,Carnegie Mellon Univ,nd8lQQIAAAAJ,data/GScholar_faloutsos_christos_cmu.json
4,Daniela Rus,Andrew (1956) and Erna Viterbi Professor of Co...,Robotics; Wireless Networks; Distributed Compu...,1071.0,0.0,67599.0,137.0,574.0,38898.0,99.0,...,https://scholar.google.com/citations?view_op=m...,https://www.eecs.mit.edu/people/daniela-rus/,(617) 258-7567,rus@csail.mit.edu,,Office: 32-374,AI & Decision-making; Computer Science,Massachusetts Institute Technology,910z20QAAAAJ,data/GScholar_daniela_rus_mit.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770,Brian A. Barsky,,,,,,,,,,...,,https://www2.eecs.berkeley.edu/Faculty/Photos/...,,barsky@berkeley.edu,,443 Soda Hall,Computer Science,Univ California Berkeley,,
771,Randy H. Katz,,,,,,,,,,...,,https://www2.eecs.berkeley.edu/Faculty/Photos/...,510-642-8778,randy@cs.berkeley.edu,,"751 Soda Hall,",Computer Science,Univ California Berkeley,,
772,Stella Yu,,,,,,,,,,...,,https://www2.eecs.berkeley.edu/Faculty/Photos/...,,,,"367 Evans Hall,",Computer Science,Univ California Berkeley,,
773,Matus Jan Telgarsky,,,,,,,,,,...,,https://ws.engr.illinois.edu/directory/viewpho...,(217) 300-4507,mjt@illinois.edu,,,Computer Science,Univ Illinois Urbana-Champaign,,


In [6]:
sql_stmt = f"""
select distinct * from df 
where scholar_id is null or scholar_id = ''
"""
df_missing = duckdb.sql(sql_stmt).df()
df_missing 

,name,affiliation,interests,num_papers,num_coauthors,citedby,hindex,i10index,citedby5y,hindex5y,...,url_picture,img_url,phone,email,cell_phone,office_address,department,school,scholar_id,file_author
0,"Ada, Anil",,,,,,,,,,...,,,(412) 268-3835,aada@andrew.cmu.edu,,6215 Gates and Hillman Centers,Computer Science,Carnegie Mellon Univ,,
1,"Berger, Daniel",,,,,,,,,,...,,,,dberger1@andrew.cmu.edu,,,Computer Science,Carnegie Mellon Univ,,
2,"Blum, Manuel",,,,,,,,,,...,,,(412) 268-3742,mblum@andrew.cmu.edu,,5013 Gates and Hillman Centers,Computer Science,Carnegie Mellon Univ,,
3,"Brookes, Stephen",,,,,,,,,,...,,,(412) 268-8820,sb21@andrew.cmu.edu,,9017 Gates and Hillman Centers,Computer Science,Carnegie Mellon Univ,,
4,"Chrysanthis, Panos",,,,,,,,,,...,,,(412) 268-4751,panos@andrew.cmu.edu,,,Computer Science,Carnegie Mellon Univ,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,Marti Hearst,,,,,,,,,,...,,https://www2.eecs.berkeley.edu/Faculty/Photos/...,510-642-8016,hearst@berkeley.edu,,"307b South Hall,",Computer Science,Univ California Berkeley,,
225,Nilah Ioannidis,,,,,,,,,,...,,https://www2.eecs.berkeley.edu/Faculty/Photos/...,,nilah@berkeley.edu,,513 Soda Hall,Computer Science,Univ California Berkeley,,
226,Alistair Sinclair,,,,,,,,,,...,,https://www2.eecs.berkeley.edu/Faculty/Photos/...,510-643-8144,sinclair@cs.berkeley.edu,,"677 Soda Hall,",Computer Science,Univ California Berkeley,,
227,Stella Yu,,,,,,,,,,...,,https://www2.eecs.berkeley.edu/Faculty/Photos/...,,,,"367 Evans Hall,",Computer Science,Univ California Berkeley,,


238 rows without scholar_id

## read extra search

In [7]:
df_x = pd.read_excel(Path(file_xlsx_extra))
df_x.fillna("", inplace=True)

In [8]:
df_x.columns 

Index(['name', 'affiliation', 'interests', 'num_papers', 'num_coauthors',
       'citedby', 'hindex', 'i10index', 'citedby5y', 'hindex5y', 'i10index5y',
       'scholar_id', 'url_author', 'url_picture', 'url_homepage',
       'file_author'],
      dtype='object')

In [9]:
df_x

,name,affiliation,interests,num_papers,num_coauthors,citedby,hindex,i10index,citedby5y,hindex5y,i10index5y,scholar_id,url_author,url_picture,url_homepage,file_author
0,"Ada, Anil",Necmettin Erbakan Üniversitesi Meram Tıp Fakül...,ANATOMİ,52,0,138,8,6,118,7,4,2jqaChUAAAAJ,https://scholar.google.com/citations?user=2jqa...,https://scholar.google.com/citations?view_op=m...,,data/GScholar_ada_anil_cmu.json
1,"Berger, Daniel",Post-Doc at Harvard University,Neuroscience; Connectomics; Electron Microscop...,63,11,4918,22,26,3738,19,25,peHbOtYAAAAJ,https://scholar.google.com/citations?user=peHb...,https://scholar.google.com/citations?view_op=m...,,data/GScholar_berger_daniel_cmu.json
2,"Blum, Manuel","Department of General Internal Medicine, Insel...",General Internal Medicine,103,0,2960,26,37,2460,23,36,VK6KPPQAAAAJ,https://scholar.google.com/citations?user=VK6K...,https://scholar.google.com/citations?view_op=m...,,data/GScholar_blum_manuel_cmu.json
3,"Brookes, Stephen",Brookhaven National Laboratory & Stony Brook U...,Climate change; Atmospheric aerosols; Atmosphe...,371,0,24581,66,142,4796,28,61,65tVwZEAAAAJ,https://scholar.google.com/citations?user=65tV...,https://scholar.google.com/citations?view_op=m...,https://wpw.bnl.gov/schwartz/,data/GScholar_brookes_stephen_cmu.json
4,"Chrysanthis, Panos","Professor of Computer Science, University of P...",Data Management; Databases; Data Streams; Mobi...,387,20,7747,39,166,1295,18,46,AQA5QtAAAAAJ,https://scholar.google.com/citations?user=AQA5...,https://scholar.google.com/citations?view_op=m...,http://panos.cs.pitt.edu/,data/GScholar_chrysanthis_panos_cmu.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,Shahar Dobzinski,Unknown affiliation,,75,0,3177,30,40,1229,20,34,teS9lHAAAAAJ,https://scholar.google.com/citations?user=teS9...,https://scholar.google.com/citations?view_op=m...,http://sites.google.com/site/dobzin/,data/GScholar_shahar_dobzinski_stanford.json
107,Marco Vassena,Utrecht University,Security; Programming Languages,15,15,194,8,7,180,7,6,q7-1wpcAAAAJ,https://scholar.google.com/citations?user=q7-1...,https://scholar.google.com/citations?view_op=m...,https://webspace.science.uu.nl/mvassena,data/GScholar_marco_vassena_stanford.json
108,Sanjit A. Seshia,"Professor of EECS, University of California, B...",Formal Methods; Design Automation; Cyber-Physi...,436,18,21771,70,194,13615,57,147,SlZavnIAAAAJ,https://scholar.google.com/citations?user=SlZa...,https://scholar.google.com/citations?view_op=m...,http://www.eecs.berkeley.edu/~sseshia,data/GScholar_sanjit_a_seshia_berkeley.json
109,Stella Yu,"Professor of EECS, University of Michigan",Computer Vision; Machine Learning; Human Vision,144,122,10531,43,88,7895,32,73,uqWkLzMAAAAJ,https://scholar.google.com/citations?user=uqWk...,https://scholar.google.com/citations?view_op=m...,http://web.eecs.umich.edu/~stellayu,data/GScholar_stella_yu_berkeley.json


In [10]:
sql_stmt = f"""
select * from df_x 
where scholar_id is null or scholar_id = ''
"""
df_x1 = duckdb.sql(sql_stmt).df()
df_x1 

,name,affiliation,interests,num_papers,num_coauthors,citedby,hindex,i10index,citedby5y,hindex5y,i10index5y,scholar_id,url_author,url_picture,url_homepage,file_author


## perform enrichment

In [11]:
df_u.shape

(775, 30)

In [12]:
col_names_str = ", ".join([c for c in df_x.columns if c != 'name'])
col_names_str

'affiliation, interests, num_papers, num_coauthors, citedby, hindex, i10index, citedby5y, hindex5y, i10index5y, scholar_id, url_author, url_picture, url_homepage, file_author'

In [13]:
sql_upd = f"""
update df_u 
set ({col_names_str}) = (
    select {col_names_str}
    from df_x 
    where name = df_u.name
)
where name in  (
    select name from df_x 
) 
"""

In [14]:
print(sql_upd)


update df_u 
set (affiliation, interests, num_papers, num_coauthors, citedby, hindex, i10index, citedby5y, hindex5y, i10index5y, scholar_id, url_author, url_picture, url_homepage, file_author) = (
    select affiliation, interests, num_papers, num_coauthors, citedby, hindex, i10index, citedby5y, hindex5y, i10index5y, scholar_id, url_author, url_picture, url_homepage, file_author
    from df_x 
    where name = df_u.name
)
where name in  (
    select name from df_x 
) 



duckdb.sql(sql_upd)

Following error was thrown:
```
NotImplementedException                   Traceback (most recent call last)
~\AppData\Local\Temp\ipykernel_1596\457050239.py in <module>
----> 1 duckdb.sql(sql_upd)

NotImplementedException: Not implemented Error: Expr of type 354 not implemented


```

Use pandas merge 

In [15]:
cols_x = [c for c in df_x.columns if c != 'name']

In [16]:
cols_old = [c for c in df_u.columns if c not in cols_x]

In [17]:
cols_old

['name',
 'phd_univ',
 'phd_year',
 'url',
 'research_area',
 'research_concentration',
 'research_focus',
 'job_title',
 'img_url',
 'phone',
 'email',
 'cell_phone',
 'office_address',
 'department',
 'school']

In [18]:
df_u_old = df_u[cols_old]

In [19]:
df_x_new = df_x.merge(df_u_old, how="inner", on="name")

In [20]:
df_x_new_2 = df_x_new[[c for c in df_u.columns]]

In [21]:
df_x_new_2

,name,affiliation,interests,num_papers,num_coauthors,citedby,hindex,i10index,citedby5y,hindex5y,...,url_picture,img_url,phone,email,cell_phone,office_address,department,school,scholar_id,file_author
0,"Ada, Anil",Necmettin Erbakan Üniversitesi Meram Tıp Fakül...,ANATOMİ,52,0,138,8,6,118,7,...,https://scholar.google.com/citations?view_op=m...,,(412) 268-3835,aada@andrew.cmu.edu,,6215 Gates and Hillman Centers,Computer Science,Carnegie Mellon Univ,2jqaChUAAAAJ,data/GScholar_ada_anil_cmu.json
1,"Berger, Daniel",Post-Doc at Harvard University,Neuroscience; Connectomics; Electron Microscop...,63,11,4918,22,26,3738,19,...,https://scholar.google.com/citations?view_op=m...,,,dberger1@andrew.cmu.edu,,,Computer Science,Carnegie Mellon Univ,peHbOtYAAAAJ,data/GScholar_berger_daniel_cmu.json
2,"Blum, Manuel","Department of General Internal Medicine, Insel...",General Internal Medicine,103,0,2960,26,37,2460,23,...,https://scholar.google.com/citations?view_op=m...,,(412) 268-3742,mblum@andrew.cmu.edu,,5013 Gates and Hillman Centers,Computer Science,Carnegie Mellon Univ,VK6KPPQAAAAJ,data/GScholar_blum_manuel_cmu.json
3,"Brookes, Stephen",Brookhaven National Laboratory & Stony Brook U...,Climate change; Atmospheric aerosols; Atmosphe...,371,0,24581,66,142,4796,28,...,https://scholar.google.com/citations?view_op=m...,,(412) 268-8820,sb21@andrew.cmu.edu,,9017 Gates and Hillman Centers,Computer Science,Carnegie Mellon Univ,65tVwZEAAAAJ,data/GScholar_brookes_stephen_cmu.json
4,"Chrysanthis, Panos","Professor of Computer Science, University of P...",Data Management; Databases; Data Streams; Mobi...,387,20,7747,39,166,1295,18,...,https://scholar.google.com/citations?view_op=m...,,(412) 268-4751,panos@andrew.cmu.edu,,,Computer Science,Carnegie Mellon Univ,AQA5QtAAAAAJ,data/GScholar_chrysanthis_panos_cmu.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Shahar Dobzinski,Unknown affiliation,,75,0,3177,30,40,1229,20,...,https://scholar.google.com/citations?view_op=m...,,,@stanford.edu,,,Computer Science,Stanford Univ,teS9lHAAAAAJ,data/GScholar_shahar_dobzinski_stanford.json
109,Marco Vassena,Utrecht University,Security; Programming Languages,15,15,194,8,7,180,7,...,https://scholar.google.com/citations?view_op=m...,,,vassena@stanford.edu,,Gates 478,Computer Science,Stanford Univ,q7-1wpcAAAAJ,data/GScholar_marco_vassena_stanford.json
110,Sanjit A. Seshia,"Professor of EECS, University of California, B...",Formal Methods; Design Automation; Cyber-Physi...,436,18,21771,70,194,13615,57,...,https://scholar.google.com/citations?view_op=m...,https://www2.eecs.berkeley.edu/Faculty/Photos/...,510-643-6968,sseshia@eecs.berkeley.edu,,"566 Cory Hall,",Computer Science,Univ California Berkeley,SlZavnIAAAAJ,data/GScholar_sanjit_a_seshia_berkeley.json
111,Stella Yu,"Professor of EECS, University of Michigan",Computer Vision; Machine Learning; Human Vision,144,122,10531,43,88,7895,32,...,https://scholar.google.com/citations?view_op=m...,https://www2.eecs.berkeley.edu/Faculty/Photos/...,,,,"367 Evans Hall,",Computer Science,Univ California Berkeley,uqWkLzMAAAAJ,data/GScholar_stella_yu_berkeley.json


In [22]:
sql_stmt = """
    select * from df_u
    where not exists (
        select 1 from df_x where df_x.name = df_u.name
    )
"""

df_u_1 = duckdb.sql(sql_stmt).df()
df_u_1

,name,affiliation,interests,num_papers,num_coauthors,citedby,hindex,i10index,citedby5y,hindex5y,...,url_picture,img_url,phone,email,cell_phone,office_address,department,school,scholar_id,file_author
0,Jiawei Han,"Abel Bliss Professor of Computer Science, Univ...",data mining; database systems; data warehousin...,1653.0,92.0,244378.0,200.0,1022.0,82746.0,117.0,...,https://scholar.google.com/citations?view_op=m...,https://ws.engr.illinois.edu/directory/viewpho...,(217) 333-6903,hanj@illinois.edu,,,Computer Science,Univ Illinois Urbana-Champaign,Kv9AbjMAAAAJ,data/GScholar_jiawei_han_uiuc.json
1,Robert Miller,"Professor, Computer Science, MIT",HCI; software engineering; crowdsourcing,1491.0,0.0,35488.0,78.0,324.0,13243.0,49.0,...,https://scholar.google.com/citations?view_op=m...,https://www.eecs.mit.edu/people/rob-miller/,(617) 324-6028,rcm@mit.edu,,Office: 32-G718,Computer Science,Massachusetts Institute Technology,e-c3R8QAAAAJ,data/GScholar_robert_miller_mit.json
2,"Kanade, Takeo",Carnegie Mellon University,Computer Vision,1302.0,0.0,147292.0,169.0,636.0,33531.0,74.0,...,https://scholar.google.com/citations?view_op=m...,,(412) 268-3016,kanade@andrew.cmu.edu,,,Computer Science,Carnegie Mellon Univ,LQ87h3sAAAAJ,data/GScholar_kanade_takeo_cmu.json
3,"Faloutsos, Christos",CMU,Data Mining; Graph Mining; Databases,1126.0,51.0,112459.0,148.0,589.0,34139.0,87.0,...,https://scholar.google.com/citations?view_op=m...,,(412) 268-1457,christos@andrew.cmu.edu,,7003 Gates and Hillman Centers,Computer Science,Carnegie Mellon Univ,nd8lQQIAAAAJ,data/GScholar_faloutsos_christos_cmu.json
4,Daniela Rus,Andrew (1956) and Erna Viterbi Professor of Co...,Robotics; Wireless Networks; Distributed Compu...,1071.0,0.0,67599.0,137.0,574.0,38898.0,99.0,...,https://scholar.google.com/citations?view_op=m...,https://www.eecs.mit.edu/people/daniela-rus/,(617) 258-7567,rus@csail.mit.edu,,Office: 32-374,AI & Decision-making; Computer Science,Massachusetts Institute Technology,910z20QAAAAJ,data/GScholar_daniela_rus_mit.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659,Victor Zue,,,,,,,,,,...,,https://www.eecs.mit.edu/people/victor-zue/,(617) 253-8513,zue@csail.mit.edu,,Office: 32-G470,AI & Decision-making; Computer Science,Massachusetts Institute Technology,,
660,Anand V Natarajan,,,,,,,,,,...,,https://www.eecs.mit.edu/people/anand-v-natara...,408-910-4116,anandn@mit.edu,,Office: 32,Computer Science,Massachusetts Institute Technology,,
661,Randy H. Katz,,,,,,,,,,...,,https://www2.eecs.berkeley.edu/Faculty/Photos/...,510-642-8778,randy@cs.berkeley.edu,,"751 Soda Hall,",Computer Science,Univ California Berkeley,,
662,Matus Jan Telgarsky,,,,,,,,,,...,,https://ws.engr.illinois.edu/directory/viewpho...,(217) 300-4507,mjt@illinois.edu,,,Computer Science,Univ Illinois Urbana-Champaign,,


In [23]:
df_12 = pd.concat([df_u_1, df_x_new_2])

In [24]:
df_12

,name,affiliation,interests,num_papers,num_coauthors,citedby,hindex,i10index,citedby5y,hindex5y,...,url_picture,img_url,phone,email,cell_phone,office_address,department,school,scholar_id,file_author
0,Jiawei Han,"Abel Bliss Professor of Computer Science, Univ...",data mining; database systems; data warehousin...,1653.0,92.0,244378.0,200.0,1022.0,82746.0,117.0,...,https://scholar.google.com/citations?view_op=m...,https://ws.engr.illinois.edu/directory/viewpho...,(217) 333-6903,hanj@illinois.edu,,,Computer Science,Univ Illinois Urbana-Champaign,Kv9AbjMAAAAJ,data/GScholar_jiawei_han_uiuc.json
1,Robert Miller,"Professor, Computer Science, MIT",HCI; software engineering; crowdsourcing,1491.0,0.0,35488.0,78.0,324.0,13243.0,49.0,...,https://scholar.google.com/citations?view_op=m...,https://www.eecs.mit.edu/people/rob-miller/,(617) 324-6028,rcm@mit.edu,,Office: 32-G718,Computer Science,Massachusetts Institute Technology,e-c3R8QAAAAJ,data/GScholar_robert_miller_mit.json
2,"Kanade, Takeo",Carnegie Mellon University,Computer Vision,1302.0,0.0,147292.0,169.0,636.0,33531.0,74.0,...,https://scholar.google.com/citations?view_op=m...,,(412) 268-3016,kanade@andrew.cmu.edu,,,Computer Science,Carnegie Mellon Univ,LQ87h3sAAAAJ,data/GScholar_kanade_takeo_cmu.json
3,"Faloutsos, Christos",CMU,Data Mining; Graph Mining; Databases,1126.0,51.0,112459.0,148.0,589.0,34139.0,87.0,...,https://scholar.google.com/citations?view_op=m...,,(412) 268-1457,christos@andrew.cmu.edu,,7003 Gates and Hillman Centers,Computer Science,Carnegie Mellon Univ,nd8lQQIAAAAJ,data/GScholar_faloutsos_christos_cmu.json
4,Daniela Rus,Andrew (1956) and Erna Viterbi Professor of Co...,Robotics; Wireless Networks; Distributed Compu...,1071.0,0.0,67599.0,137.0,574.0,38898.0,99.0,...,https://scholar.google.com/citations?view_op=m...,https://www.eecs.mit.edu/people/daniela-rus/,(617) 258-7567,rus@csail.mit.edu,,Office: 32-374,AI & Decision-making; Computer Science,Massachusetts Institute Technology,910z20QAAAAJ,data/GScholar_daniela_rus_mit.json
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,Shahar Dobzinski,Unknown affiliation,,75,0,3177,30,40,1229,20,...,https://scholar.google.com/citations?view_op=m...,,,@stanford.edu,,,Computer Science,Stanford Univ,teS9lHAAAAAJ,data/GScholar_shahar_dobzinski_stanford.json
109,Marco Vassena,Utrecht University,Security; Programming Languages,15,15,194,8,7,180,7,...,https://scholar.google.com/citations?view_op=m...,,,vassena@stanford.edu,,Gates 478,Computer Science,Stanford Univ,q7-1wpcAAAAJ,data/GScholar_marco_vassena_stanford.json
110,Sanjit A. Seshia,"Professor of EECS, University of California, B...",Formal Methods; Design Automation; Cyber-Physi...,436,18,21771,70,194,13615,57,...,https://scholar.google.com/citations?view_op=m...,https://www2.eecs.berkeley.edu/Faculty/Photos/...,510-643-6968,sseshia@eecs.berkeley.edu,,"566 Cory Hall,",Computer Science,Univ California Berkeley,SlZavnIAAAAJ,data/GScholar_sanjit_a_seshia_berkeley.json
111,Stella Yu,"Professor of EECS, University of Michigan",Computer Vision; Machine Learning; Human Vision,144,122,10531,43,88,7895,32,...,https://scholar.google.com/citations?view_op=m...,https://www2.eecs.berkeley.edu/Faculty/Photos/...,,,,"367 Evans Hall,",Computer Science,Univ California Berkeley,uqWkLzMAAAAJ,data/GScholar_stella_yu_berkeley.json


add a new column to tag those faculties without Google Scholar profiles

In [25]:
from datetime import datetime
str(datetime.now())

'2023-09-10 22:50:05.172383'

In [26]:
def tag_missing_profile(row):
    return datetime.now().strftime('%Y-%m-%d') if row["scholar_id"] == "" else ""

In [27]:
df_12["not_found_date"] = df_12.apply(tag_missing_profile, axis=1)

In [32]:
sql_stmt = """
    select not_found_date, count(*) from df_12
    group by not_found_date
"""

df_12_count = duckdb.sql(sql_stmt).df()
df_12_count

,not_found_date,count_star()
0,,659
1,2023-09-10,118


## write out result

In [33]:
dt = datetime.now().strftime('%Y-%m-%d')

In [34]:
file_xlsx = f"data/cs_faculty_top6-{dt}.xlsx"
writer = pd.ExcelWriter(Path(file_xlsx), engine='xlsxwriter')
df_12.to_excel(writer, sheet_name="CS-Scholars", index=False)
writer.close()